In [ ]:
#| default_exp oauth

# OAuth

- eval: false
- skip_exec: true

This provides the basic scaffolding for handling OAuth. It is not yet thoroughly tested.

In [ ]:
#| export
from fasthtml.common import *
from oauthlib.oauth2 import WebApplicationClient
from httpx import get, post
import secrets

In [ ]:
#| export
class _AppClient(WebApplicationClient):
    def __init__(self, client_id, client_secret, redirect_uri, code=None, scope=None, **kwargs):
        super().__init__(client_id, code=code, scope=scope, **kwargs)
        self.client_secret,self.redirect_uri = client_secret,redirect_uri

In [ ]:
#| export
class GoogleAppClient(_AppClient):
    "A `WebApplicationClient` for Google oauth2"
    base_url = "https://accounts.google.com/o/oauth2/v2/auth"
    token_url = "https://www.googleapis.com/oauth2/v4/token"
    info_url = "https://www.googleapis.com/oauth2/v3/userinfo"
    id_key = 'sub'
    
    def __init__(self, client_id, client_secret, redirect_uri=None, redirect_uris=None, code=None, scope=None, **kwargs):
        if redirect_uris and not redirect_uri: redirect_uri = redirect_uris[0]
        scope_pre = "https://www.googleapis.com/auth/userinfo"
        if not scope: scope=["openid", f"{scope_pre}.email", f"{scope_pre}.profile"]
        super().__init__(client_id, client_secret, redirect_uri, code=code, scope=scope, **kwargs)

In [ ]:
#| export
class GitHubAppClient(_AppClient):
    "A `WebApplicationClient` for GitHub oauth2"
    base_url = "https://github.com/login/oauth/authorize"
    token_url = "https://github.com/login/oauth/access_token"
    info_url = "https://api.github.com/user"
    id_key = 'id'

    def __init__(self, client_id, client_secret, redirect_uri, code=None, scope=None, **kwargs):
        if not scope: scope="user"
        super().__init__(client_id, client_secret, redirect_uri, code=code, scope=scope, **kwargs)

In [ ]:
#| export
class HuggingFaceClient(_AppClient):
    "A `WebApplicationClient` for HuggingFace oauth2"

    base_url = "https://huggingface.co/oauth/authorize"
    token_url = "https://huggingface.co/oauth/token"
    info_url = "https://huggingface.co/oauth/userinfo"
    id_key = 'sub'
    
    def __init__(self, client_id, client_secret, redirect_uri=None, redirect_uris=None, code=None, scope=None, state=None, **kwargs):
        if redirect_uris and not redirect_uri: redirect_uri = redirect_uris[0]
        if not scope: scope=["openid","profile"]
        if not state: state=secrets.token_urlsafe(16)
        super().__init__(client_id, client_secret, redirect_uri, code=code, scope=scope, state=state, **kwargs)

In [ ]:
#| export
@patch
def login_link(self:WebApplicationClient, scope=None):
    "Get a login link for this client"
    if not scope: scope=self.scope
    return self.prepare_request_uri(self.base_url, self.redirect_uri, scope)

Generating a login link that sends the user to the OAuth provider is done with `client.login_link()`:

In [ ]:
host,port = 'localhost',8000
redirect_uri = f"http://{host}:{port}/redirect"
client = GoogleAppClient("YOUR_CLIENT_ID","YOUR_CLIENT_SECRET",redirect_uri)
print(client.login_link())

https://accounts.google.com/o/oauth2/v2/auth?response_type=code&client_id=YOUR_CLIENT_ID&redirect_uri=http%3A%2F%2Flocalhost%3A8000%2Fredirect&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.profile


In [ ]:
#| export
@patch
def login_link_with_state(self:WebApplicationClient, scope=None, state=None):
    "Get a login link for this client"
    if not scope: scope=self.scope
    if not state: state=self.state
    return self.prepare_request_uri(self.base_url, self.redirect_uri, scope, state)

It can sometimes be useful to pass state to the OAuth provider, so that when the user returns you can pick up where they left off. This can be done by using the `login_link_with_state` function with a `state` parameter:

TODO: do all providers support this the same way? This is only tested for HF atm.

In [ ]:
client = HuggingFaceClient("YOUR_CLIENT_ID","YOUR_CLIENT_SECRET",redirect_uri)
print(client.login_link_with_state(state="test_state"))

https://huggingface.co/oauth/authorize?response_type=code&client_id=YOUR_CLIENT_ID&redirect_uri=http%3A%2F%2Flocalhost%3A8000%2Fredirect&scope=openid+profile&state=test_state


In [ ]:
#| export
@patch
def parse_response(self:_AppClient, code):
    "Get the token from the oauth2 server response"
    payload = dict(code=code, redirect_uri=self.redirect_uri, client_id=self.client_id,
                   client_secret=self.client_secret, grant_type='authorization_code')
    self.parse_request_body_response(post(self.token_url, json=payload).text)

In [ ]:
#| export
@patch
def get_info(self:_AppClient):
    "Get the info for authenticated user"
    headers = {'Authorization': f'Bearer {self.token["access_token"]}'}
    return get(self.info_url, headers=headers).json()

In [ ]:
#| export
@patch
def retr_info(self:_AppClient, code):
    "Combines `parse_response` and `get_info`"
    self.parse_response(code)
    return self.get_info()

In [ ]:
#| export
@patch
def retr_id(self:_AppClient, code):
    "Call `retr_info` and then return id/subscriber value"
    return self.retr_info(code)[self.id_key]

After logging in via the provider, the user will be redirected back to the supplied redirect URL. The request to this URL will contain a `code` parameter, which is used to get an access token and fetch the user's profile information. See TODO link explanation for a worked example. You can either:

- use client.retr_info(code) to get all the profile information, or
- use client.retr_id(code) to get just the user's ID.

After either of these calls, you can also access the access token (used to revoke access, for example) with `client.token["access_token"]`.

# Export -

In [ ]:
#|hide
import nbdev; nbdev.nbdev_export()